In [3]:
from keras import backend as K
K.set_image_data_format('channels_first')
from keras.models import model_from_json
import cv2
import numpy as np
import os
from sklearn.cluster import KMeans

Using TensorFlow backend.


In [5]:
def load_facenet():
    """
    loads a saved pretrained model from a json file
    :return:
    """
    # load json and create model
    json_file = open('a5/FRmodel.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    FRmodel = model_from_json(loaded_model_json)

    # load weights into new model
    FRmodel.load_weights("a5/FRmodel.h5")
    print("Loaded model from disk")

    return FRmodel

In [6]:

def img_to_encoding(img1, model):
    """
    returns 128-dimensional face embedding for input image
    :param img1:
    :param model:
    :return:
    """
    img = img1[...,::-1]
    img = np.around(np.transpose(img, (2,0,1))/255.0, decimals=12)
    x_train = np.array([img])
    embedding = model.predict_on_batch(x_train)
    return embedding

In [7]:
def load_dataset():

    if not os.path.exists('a5/saved_faces/'):
        os.makedirs('a/5saved_faces')

    a = np.load('a5/faces.npy')

    for i in range(a.shape[0]):

        img = a[i][..., ::-1]
        img = cv2.resize(img, (96, 96))
        cv2.imwrite("a5/saved_faces/face_image_"+str(i)+".jpg", img)

In [19]:
def compute_Kmeans(X):
    kmeans = KMeans(n_clusters=6).fit(X)
    labels = kmeans.labels_
    result = {}
    result[-1] = "no matching"
    result[0] = []
    result[1] = []
    result[2] = []
    result[3] = []
    result[4] = []
    result[5] = []
    for i in range(len(labels)):
        result[labels[i]].append(i)
    print(result)
    visual_word = kmeans.cluster_centers_
    print(visual_word)
    return result,visual_word

In [18]:
def matching(img,model,visual_word):
    ref_embedding = img_to_encoding(img,model)
    max_sim = 0
    max_idx = 0
    for i in range(len(visual_word)):
        sim = np.dot(visual_word[i],ref_embedding.T)/np.linalg.norm(ref_embedding)*np.linalg.norm(visual_word[i])
        if sim > max_sim:
            max_sim = sim
            max_idx = i
    if max_sim < 0.8:
        return -1
    return max_idx
            
        

In [20]:
load_dataset()
model = load_facenet()
X = []
for i in range(200):
    img1 = cv2.imread("a5/saved_faces/face_image_"+str(i)+".jpg")
    embedding = img_to_encoding(img1,model).tolist()
    X.append(embedding)
    np.save("a5/saved_faces/face_image_"+str(i),embedding)
X = np.array(X).reshape(200,128)
print(X.shape)
inverted_dict, visual_word = compute_Kmeans(X)
np.save("a5/saved_faces/visual_word",visual_word)
img0 = cv2.imread("a5/input_faces/face_image_97.jpg")
img0 = cv2.resize(img0, (96, 96))
matching0 = matching(img0,model,visual_word)
print("face 97 : "+str(inverted_dict[matching0]))

img1 = cv2.imread("a5/input_faces/face_image_107.jpg")
img1 = cv2.resize(img1, (96, 96))
matching1 = matching(img1,model,visual_word)
print("face 107 : "+str(inverted_dict[matching1]))

img2 = cv2.imread("a5/input_faces/face_image_109.jpg")
img2 = cv2.resize(img2, (96, 96))
matching2 = matching(img2,model,visual_word)
print("face 109 : "+str(inverted_dict[matching2]))

img3 = cv2.imread("a5/input_faces/face_image_116.jpg")
img3 = cv2.resize(img3, (96, 96))
matching3 = matching(img3,model,visual_word)
print("face 116 : "+str(inverted_dict[matching3]))

img4 = cv2.imread("a5/input_faces/face_image_119.jpg")
img4 = cv2.resize(img4, (96, 96))
matching4 = matching(img4,model,visual_word)
print("face 119 : "+str(inverted_dict[matching4]))

img5 = cv2.imread("a5/input_faces/face_image_126.jpg")
img5 = cv2.resize(img5, (96, 96))
matching5 = matching(img5,model,visual_word)
print("face 126 : "+str(inverted_dict[matching5]))

img6 = cv2.imread("a5/input_faces/face_image_600.jpg")
img6 = cv2.resize(img6, (96, 96))
matching6 = matching(img6,model,visual_word)
print("face 600 : "+str(inverted_dict[matching6]))

Loaded model from disk
(200, 128)
{-1: 'no matching', 0: [8, 17, 18, 20, 27, 28, 37, 40, 44, 55, 60, 67, 74, 76, 82, 86, 90, 92, 93, 99, 110, 118, 121, 126, 127, 128, 129, 134, 137, 141, 147, 151, 153, 155, 159, 163, 167, 173, 181, 185, 196], 1: [0, 1, 2, 5, 7, 11, 14, 16, 23, 31, 36, 48, 52, 56, 61, 66, 70, 79, 81, 83, 88, 101, 112, 116, 148, 149, 154, 156, 162, 172, 177, 184, 190, 198, 199], 2: [32, 33, 45, 54, 57, 62, 77, 80, 84, 89, 98, 100, 105, 106, 108, 117, 119, 122, 135, 140, 145, 157, 161, 165, 180, 186, 187, 189, 192, 194, 197], 3: [9, 10, 12, 13, 26, 34, 39, 42, 43, 53, 59, 63, 64, 65, 68, 72, 73, 75, 85, 91, 94, 97, 102, 103, 107, 111, 113, 132, 136, 142, 170, 178, 183, 191], 4: [3, 21, 24, 25, 29, 30, 35, 41, 46, 49, 51, 71, 78, 104, 114, 115, 120, 123, 124, 125, 139, 146, 150, 152, 164, 166, 168, 169, 171, 174, 176, 182, 193, 195], 5: [4, 6, 15, 19, 22, 38, 47, 50, 58, 69, 87, 95, 96, 109, 130, 131, 133, 138, 143, 144, 158, 160, 175, 179, 188]}
[[ 0.06931463  0.03432185 

face 107 : [4, 6, 15, 19, 22, 38, 47, 50, 58, 69, 87, 95, 96, 109, 130, 131, 133, 138, 143, 144, 158, 160, 175, 179, 188]
face 109 : [32, 33, 45, 54, 57, 62, 77, 80, 84, 89, 98, 100, 105, 106, 108, 117, 119, 122, 135, 140, 145, 157, 161, 165, 180, 186, 187, 189, 192, 194, 197]
face 116 : [8, 17, 18, 20, 27, 28, 37, 40, 44, 55, 60, 67, 74, 76, 82, 86, 90, 92, 93, 99, 110, 118, 121, 126, 127, 128, 129, 134, 137, 141, 147, 151, 153, 155, 159, 163, 167, 173, 181, 185, 196]
face 119 : [9, 10, 12, 13, 26, 34, 39, 42, 43, 53, 59, 63, 64, 65, 68, 72, 73, 75, 85, 91, 94, 97, 102, 103, 107, 111, 113, 132, 136, 142, 170, 178, 183, 191]
face 126 : [3, 21, 24, 25, 29, 30, 35, 41, 46, 49, 51, 71, 78, 104, 114, 115, 120, 123, 124, 125, 139, 146, 150, 152, 164, 166, 168, 169, 171, 174, 176, 182, 193, 195]
face 600 : no matching


C:\Users\zhang\Anaconda3\envs\python3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


AttributeError: 'numpy.ndarray' object has no attribute 'predict_on_batch'